 > # Online shoppers intention classification

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> # Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import hamming_loss
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import jaccard_score
from sklearn.metrics import log_loss
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import zero_one_loss
from sklearn.metrics import brier_score_loss
from sklearn.metrics import balanced_accuracy_score

> # Loading the data 

In [4]:
df = pd.read_csv("C:\Users\dell\Desktop\online_shoppers_intention.csv")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-4-2480efce1ccc>, line 1)

> # EDA and preprocessing 

In [ ]:
df.shape


In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.isna().any()

In [ ]:
df.isna().sum()

In [ ]:
np.where(df.applymap(lambda x: x == 'nan'))
np.where(pd.isnull(df))
df.loc[1065][7]

In [ ]:
df['Administrative'].mode()
df['Administrative_Duration'].mode()
df['Informational'].mode()
df['Informational_Duration'].mode()
df.mode()

* > Using np.nan to replace those null values.

In [ ]:
df = df.replace(np.nan,0)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df['Month'].unique()

In [ ]:
df.corr()

In [ ]:
df['Weekend'] = df['Weekend'].replace(False,0)
df['Weekend'] = df['Weekend'].replace(True,1)

In [ ]:
df['Weekend'] = df['Weekend'].astype(int)

In [ ]:
df['Revenue'] = df['Revenue'].replace(False,0)
df['Revenue'] = df['Revenue'].replace(True,1)

In [ ]:
df['Revenue'] = df['Revenue'].astype(int)

In [ ]:
df['Revenue']

In [ ]:
# df = df.drop(columns=['Month'])

In [ ]:
df.info()

In [ ]:
y = df["Revenue"]

In [ ]:
X = df.drop(columns='Revenue')

In [ ]:
X.head()

In [ ]:
df.info()

In [ ]:
df['VisitorType'].unique()

In [ ]:
X = pd.get_dummies(X,drop_first=True)

In [ ]:
X.shape

In [ ]:
df.describe()

In [ ]:
df['Revenue'].value_counts()
1908/(1908+10422)

> # Normalisation

In [ ]:
from sklearn import preprocessing
# from sklearn.preprocessing import Normalizer
X = preprocessing.normalize(X, norm='l2')

> # Train-test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
y_test.value_counts()

> # GENERATE MODEL REPORT

In [ ]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print("F1 Score = " ,f1_score(y_actual, y_predicted))
    print("Cohen Kappa score= ",cohen_kappa_score(y_actual,y_predicted))
    print("Hamming Loss = ",hamming_loss(y_actual,y_predicted))
    print("Jaccard Score = ",jaccard_score(y_actual, y_predicted))
#     print("Log Loss = ",log_loss(y_actual, y_predicted))
    print("Matthews correlation coefficient = ", matthews_corrcoef(y_actual, y_predicted))
    print("Zero one loss = ",zero_one_loss(y_actual, y_predicted))
#     print("brier_score_loss  = ",brier_score_loss(y_actual, y_predicted))
    print("Balanced accuracy = ",balanced_accuracy_score(y_actual, y_predicted))
    
    
    pass


> # Generate auc_roc_curve

In [ ]:
def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test,  y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
    plt.legend(loc=4)
    plt.show()
    pass


> # USING STATSMODELS TO CALCULATE THE SUMMARY

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y_train,X_train)
result=logit_model.fit()
result.summary()

1.  > # LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0,class_weight = "balanced")
classifier.get_params()

> # Grid Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV
grid={"C":np.logspace(-3,3,5,7), "penalty":["l1","l2"],"max_iter":[100,200,300,400,500]}# l1 lasso l2 ridge
grid_cv=GridSearchCV(classifier,grid,cv=10,verbose=0)
grid_cv.fit(X_train,y_train)
print("tuned hpyerparameters :(best parameters) ",grid_cv.best_params_)
print("accuracy :",grid_cv.best_score_)


In [ ]:
classifier = LogisticRegression(random_state = 0,class_weight = "balanced",C=31.622776601683793,max_iter=200,penalty='l2')
classifier.fit(X_train,y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv =10,verbose=0)
accuracies.std()
accuracies.mean()

> # Changing the class weights

In [ ]:
unique_classes = list(y.unique())
unique_classes

In [ ]:
out_dict = {}
for classes in unique_classes:
    out_dict[classes] = len(y_train)/(len(unique_classes)*y_train.value_counts(classes))
out_dict

In [ ]:
from sklearn.utils import class_weight

In [ ]:
class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)

In [ ]:
pd.crosstab(pd.Series(y_pred,name='Predicted'),
           pd.Series(y_test,name="Actual"))

In [ ]:
generate_model_report(y_test,y_pred)

In [ ]:
generate_auc_roc_curve(classifier,X_test)

In [ ]:
y_pred_proba = classifier.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
plt.legend(loc=4)
plt.show()


> # Finding the optimal threshold 

In [ ]:
optimal_idx = np.argmax(tpr-fpr)
optimal_threshold = thresholds[optimal_idx]
optimal_threshold

In [ ]:
tpr

In [ ]:
fpr

In [ ]:
tpr-fpr

In [ ]:
np.argmax(tpr-fpr)

In [ ]:
optimal_idx

In [ ]:
thresholds

# argmax of tpr-fpr is the optimal cut off for the decision boundary threshold

In [ ]:
thresholds[204]

In [ ]:
y_test.value_counts()

# Using the optimal threshold 

In [ ]:
y_prob = classifier.predict_proba(X_test)
pd.DataFrame(classifier.predict_proba(X_test), columns=classifier.classes_)


In [ ]:
y_prob

# Making a y_pp for assigning labels according to the optimal threshold

In [ ]:
y_pp = []
for i in range(len(y_prob)):
    if y_prob[i][0]>optimal_threshold:
        y_pp.append(0)
    else:
        y_pp.append(1)


# Evaluating the new threshold, no need to calc auc roc again as it depends on the model.

In [ ]:
generate_model_report(y_test,y_pp)

# Comparing predicted values which we got from 0.3 change in the threshold value. 

In [ ]:
generate_model_report(y_pred,y_pp)

# GridSearch for class weights.

In [ ]:
weights = np.linspace(0.05, 0.95, 20)
gsc = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid={"C":np.logspace(-3,3,5,7), "penalty":["l1","l2"],"max_iter":[100,200,300,400,500],
        'class_weight': [{0: x, 1: 1.0-x} for x in weights]
    },refit=True,
    
    cv=10
)

gsc.fit(X_train,y_train)

In [ ]:
gsc.best_params_

In [ ]:
gsc.get_params

In [ ]:
clf = LogisticRegression(C= 1000.0,
 class_weight= {0: 0.23947368421052628, 1: 0.7605263157894737},
 max_iter= 500,
 penalty= 'l2')
clf.fit(X_train,y_train)
clf.get_params()

In [ ]:
y_prob2 = clf.predict_proba(X_test)
y_prob2

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,  y_prob2[:,1])
auc = roc_auc_score(y_test, y_prob2[:,1])
plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
opt = np.argmax(tpr-fpr)
opt_th = thresholds[opt]
opt_th

In [ ]:
y_pp2 = []
for i in range(len(y_prob2)):
    if y_prob2[i][0]>opt_th:
        y_pp2.append(0)
    else:
        y_pp2.append(1)

In [ ]:
y_pred2 = clf.predict(X_test)

# Comparing different y_pred

In [ ]:
#Previous model
generate_model_report(y_test,y_pred)

In [ ]:
#Previous Model
generate_model_report(y_test,y_pp)

In [ ]:
generate_model_report(y_test,y_pred2)

In [ ]:
generate_model_report(y_test,y_pp2)

In [ ]:
clf.coef_

In [ ]:
clf.intercept_

In [ ]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = clf, X = X_train, y = y_train, cv =10,verbose=0)
accuracies.std()
accuracies.mean()

In [ ]:
max(0.8932467924807487,0.8935508466042323)

# ROC CURVE ON THE TRAINING DATA

In [ ]:
y_pred_proba = clf.predict_proba(X_train)[:, 1]
fpr, tpr, thresholds = roc_curve(y_train,  y_pred_proba)
auc = roc_auc_score(y_train, y_pred_proba)
plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
plt.legend(loc=4)
plt.show()


# Calculating threshold based on training data

In [ ]:
thresholds[np.argmax(tpr-fpr)]

In [ ]:
generate_auc_roc_curve(clf,X_test)

# Model without fitting the intercept

In [ ]:
clf2 = LogisticRegression(C= 1000,
 class_weight= {0: 0.23947368421052628, 1: 0.7605263157894737},
 max_iter= 500,
 penalty= 'l2',fit_intercept = False)
clf2.fit(X_train,y_train)
y_pred3 = clf2.predict(X_test)

In [ ]:
generate_model_report(y_test,y_pred3)

In [ ]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = clf2, X = X_train, y = y_train, cv =10,verbose=0)
accuracies.std()
accuracies.mean()

# The evaluation metrics doesn't change much when the intercept is not fitted.

In [ ]:
generate_auc_roc_curve(clf2,X_test)

# Learning Curve

In [ ]:
from yellowbrick.model_selection import LearningCurve
visualizer = LearningCurve(
    clf, cv=10, scoring='f1',verbose = 0
)

visualizer.fit(X_train, y_train)        # Fit the data to the visualizer
visualizer.show()

# PRECISION-RECALL CURVE

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

In [ ]:
disp = plot_precision_recall_curve(clf, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: ')

In [ ]:
plt.plot(X_test,y_prob2)

In [ ]:
coef = list(clf2.coef_)
m = max(coef)
coef.index(m)
coef

In [ ]:
# df['TrafficType'].unique()

In [ ]:
# df['Region'].unique()

# COHEN KAPPA SCORE

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_test, y_pred2)

# classification_report

In [ ]:
from sklearn.metrics import classification_report
C=['class 0', 'class 1']
print(classification_report(y_test, y_pred2, target_names=C))

In [ ]:
len(y_test)

# Plotting Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(clf, X_test, y_test)  # doctest: +SKIP
plt.show()  # doctest: +SKIP

# HAMMING LOSS

In [ ]:
from sklearn.metrics import hamming_loss
hamming_loss(y_test, y_pred2)

# Average precision, recall score

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
precision, recall, threshold = precision_recall_curve(y_test, y_pred2)

In [ ]:
precision
recall
threshold

In [ ]:
average_precision_score(y_test, y_pred2)

# f1 and fbeta scores

In [ ]:
from sklearn import metrics
metrics.precision_score(y_test, y_pred2)
metrics.recall_score(y_test, y_pred2)
metrics.f1_score(y_test, y_pred2)
metrics.fbeta_score(y_test, y_pred2, beta=0.5)
metrics.fbeta_score(y_test, y_pred2, beta=1)
metrics.fbeta_score(y_test, y_pred2, beta=2)
metrics.precision_recall_fscore_support(y_test, y_pred2, beta=0.5)

# JACCARD SCORE

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test,y_pred2)

# HINGE LOSS

In [ ]:
from sklearn.metrics import hinge_loss
hinge_loss(y_test,y_pred2)

# LOG LOSS

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, y_prob2)

# Matthews correlation coefficient

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, y_pred2)

# ZERO ONE LOSS

In [ ]:
from sklearn.metrics import zero_one_loss
zero_one_loss(y_test, y_pred2)

# BRIER SCORE LOSS

In [ ]:
from sklearn.metrics import brier_score_loss
brier_score_loss(y_test, y_prob2[:,1])
# brier_score_loss(y_test, 1-y_prob2[:,1])

# balanced_accuracy_score

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred2)

In [ ]:
clf.decision_function(X_train)

# NAIVE BAYES

# Gaussian NB

In [ ]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)

In [ ]:
nb.get_params()

# Probability of each class (prior).



In [ ]:
nb.class_prior_

In [ ]:
nb.class_count_

In [ ]:
nb.classes_

# Absolute additive value to variances-epsilon

In [ ]:
nb.epsilon_

# Theta: mean of each feature per class

In [ ]:
nb.theta_

In [ ]:
y_nb = nb.predict(X_test)

In [ ]:
y_nb_prob = nb.predict_proba(X_test)
y_nb_prob

# EVALUATION

In [ ]:
nb.score(X_test,y_test)

# classification_report

In [ ]:
from sklearn.metrics import classification_report
C=['class 0', 'class 1']
print(classification_report(y_test, y_nb, target_names=C))

# k-Fold Cross Validation

In [ ]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = nb, X = X_train, y = y_train, cv =10,verbose=0)
accuracies.std()
accuracies.mean()

# Learning Curve

In [ ]:
from yellowbrick.model_selection import LearningCurve
visualizer = LearningCurve(
    nb, cv=10, scoring='f1',verbose = 0
)

visualizer.fit(X_train, y_train)        # Fit the data to the visualizer
visualizer.show()

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(nb, X_test, y_test)  # doctest: +SKIP
plt.show()  # doctest: +SKIP

# PRECISION-RECALL CURVE

In [ ]:
disp = plot_precision_recall_curve(nb, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: ')

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
precision, recall, threshold = precision_recall_curve(y_test, y_nb)


# OTHER EVALUATION METRICS

In [ ]:
from sklearn import metrics
metrics.precision_score(y_test, y_nb)
metrics.recall_score(y_test, y_nb)
metrics.f1_score(y_test, y_nb)
metrics.fbeta_score(y_test, y_nb, beta=0.5)
metrics.fbeta_score(y_test, y_nb, beta=1)
metrics.fbeta_score(y_test, y_nb, beta=2)
metrics.precision_recall_fscore_support(y_test, y_nb, beta=0.5)

In [ ]:
from sklearn.metrics import hamming_loss
hamming_loss(y_test, y_nb)

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test,y_nb)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, y_nb)

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_test, y_nb)

In [ ]:
from sklearn.metrics import zero_one_loss
zero_one_loss(y_test, y_nb)

In [ ]:
from sklearn.metrics import brier_score_loss
brier_score_loss(y_test, y_nb_prob[:,1])
brier_score_loss(y_test, 1-y_nb_prob[:,1])

In [ ]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_nb)

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_test, y_nb)

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). 

In [ ]:
# from sklearn.naive_bayes import MultinomialNB
# nb2 = MultinomialNB()
# nb2.fit(X_train,y_train)
# y_nb2 = nb2.predict(X_test)
#IT WON'T WORK AS THE VALUES HAVE NEGATIVE VALUES TOO.


# Test with permutations for the significance of a classification score
 In order to test if a classification score is significative a technique in repeating the classification procedure after randomizing, permuting, the labels. The p-value is then given by the percentage of runs for which the score obtained is greater than the classification score obtained in the first place.



In [ ]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(10)

In [ ]:
from sklearn.model_selection import permutation_test_score
score, permutation_scores, pvalue = permutation_test_score(
    GaussianNB(), X_train, y_train, scoring="accuracy", cv=cv, n_permutations=100, n_jobs=1)
print("Classification score %s (pvalue : %s)" % (score, pvalue))

In [ ]:
# View histogram of permutation scores
plt.hist(permutation_scores, 20, label='Permutation scores',
         edgecolor='black')
ylim = plt.ylim()
# BUG: vlines(..., linestyle='--') fails on older versions of matplotlib
# plt.vlines(score, ylim[0], ylim[1], linestyle='--',
#          color='g', linewidth=3, label='Classification Score'
#          ' (pvalue %s)' % pvalue)
# plt.vlines(1.0 / n_classes, ylim[0], ylim[1], linestyle='--',
#          color='k', linewidth=3, label='Luck')
plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score'
         ' (pvalue %s)' % pvalue)
plt.plot(2 * [1. / 2], ylim, '--k', linewidth=3, label='Luck')

plt.ylim(ylim)
plt.legend()
plt.xlabel('Score')
plt.show()

# For comparision with Logistic Regression

In [ ]:
from sklearn.model_selection import permutation_test_score
score, permutation_scores, pvalue = permutation_test_score(
    clf, X, y, scoring="accuracy", cv=20, n_permutations=100, n_jobs=1)
print("Classification score %s (pvalue : %s)" % (score, pvalue))

In [ ]:
# View histogram of permutation scores
plt.hist(permutation_scores, 30, label='Permutation scores',
         edgecolor='black')
ylim = plt.ylim()
# BUG: vlines(..., linestyle='--') fails on older versions of matplotlib
# plt.vlines(score, ylim[0], ylim[1], linestyle='--',
#          color='g', linewidth=3, label='Classification Score'
#          ' (pvalue %s)' % pvalue)
# plt.vlines(1.0 / n_classes, ylim[0], ylim[1], linestyle='--',
#          color='k', linewidth=3, label='Luck')
plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score'
         ' (pvalue %s)' % pvalue)
plt.plot(2 * [1. / 2], ylim, '--k', linewidth=3, label='Luck')

plt.ylim(ylim)
plt.legend()
plt.xlabel('Score')
plt.show()

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3,weights = 'distance')
knn.fit(X_train,y_train)

In [ ]:
knn.get_params()

In [ ]:
y_knn = knn.predict(X_test)

In [ ]:
y_knn_prob = knn.predict_proba(X_test)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(knn, X_test, y_test)  # doctest: +SKIP
plt.show()  # doctest: +SKIP

In [ ]:
from sklearn.metrics import classification_report
C=['class 0', 'class 1']
print(classification_report(y_test, y_knn, target_names=C))

In [ ]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = knn, X = X_train, y = y_train, cv =10,verbose=0)
accuracies.std()
accuracies.mean()
#This can be misleading as it didn't change much when we changed the n_neighbours parameter from 2 to 200.

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_test, y_knn)

In [ ]:
jaccard_score(y_test,y_knn)

In [ ]:
hamming_loss(y_test,y_knn)

In [ ]:
log_loss(y_test,y_knn_prob)

In [ ]:
matthews_corrcoef(y_test, y_knn)

In [ ]:
zero_one_loss(y_test, y_knn)

In [ ]:
brier_score_loss(y_test,y_knn_prob[:,1])

In [ ]:
balanced_accuracy_score(y_test,y_knn)

In [ ]:
generate_model_report(y_test,y_knn)

# permutation_test_score for knn : score and p value

In [ ]:
from sklearn.model_selection import permutation_test_score
score, permutation_scores, pvalue = permutation_test_score(
    knn, X, y, scoring="accuracy", cv=20, n_permutations=100, n_jobs=1)
print("Classification score %s (pvalue : %s)" % (score, pvalue))

In [ ]:
# View histogram of permutation scores
plt.hist(permutation_scores, 30, label='Permutation scores',
         edgecolor='black')
ylim = plt.ylim()
# BUG: vlines(..., linestyle='--') fails on older versions of matplotlib
# plt.vlines(score, ylim[0], ylim[1], linestyle='--',
#          color='g', linewidth=3, label='Classification Score'
#          ' (pvalue %s)' % pvalue)
# plt.vlines(1.0 / n_classes, ylim[0], ylim[1], linestyle='--',
#          color='k', linewidth=3, label='Luck')
plt.plot(2 * [score], ylim, '--g', linewidth=3,
         label='Classification Score'
         ' (pvalue %s)' % pvalue)
plt.plot(2 * [1. / 2], ylim, '--k', linewidth=3, label='Luck')

plt.ylim(ylim)
plt.legend()
plt.xlabel('Score')
plt.show()

#  DIFFERENT SCORING METRICS for GRIDSEARCHCV

In [ ]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

# GRID SEARCH FOR DIFFERENT SCORING METRICS

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={'weights':['distance','uniform'], 'n_neighbors':[3,5,7,9,12,15,20,25]}
kn = GridSearchCV(knn, parameters,scoring='recall', cv=5)
kn.fit(X_train,y_train)
kn.best_params_
params = kn.best_params_
kn_grid = KNeighborsClassifier()
kn_grid.set_params(**params)
kn_grid.fit(X_train,y_train)
y_grid = kn_grid.predict(X_test)
generate_model_report(y_test,y_grid)


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={'weights':['distance','uniform'], 'n_neighbors':[3,5,7,9,12,15,20,25]}
kn = GridSearchCV(knn, parameters,scoring='f1', cv=5)
kn.fit(X_train,y_train)
params = kn.best_params_
kn_grid = KNeighborsClassifier()
kn_grid.set_params(**params)
kn_grid.fit(X_train,y_train)

y_grid = kn_grid.predict(X_test)
generate_model_report(y_test,y_grid)


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={'weights':['distance','uniform'], 'n_neighbors':[3,5,7,9,12,15,20,25]}
kn = GridSearchCV(knn, parameters,scoring='precision', cv=5)
kn.fit(X_train,y_train)
kn.best_score_
params = kn.best_params_
kn_grid = KNeighborsClassifier()
kn_grid.set_params(**params)
kn_grid.fit(X_train,y_train)

y_grid = kn_grid.predict(X_test)
generate_model_report(y_test,y_grid)


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={'weights':['distance','uniform'], 'n_neighbors':[3,5,7,9,12,15,20,25]}
kn = GridSearchCV(knn, parameters,scoring= 'balanced_accuracy',
 cv=5)
kn.fit(X_train,y_train)
kn.best_params_
kn.best_score_
params = kn.best_params_
kn_grid = KNeighborsClassifier()
kn_grid.set_params(**params)
kn_grid.fit(X_train,y_train)

y_grid = kn_grid.predict(X_test)
generate_model_report(y_test,y_grid)


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={'weights':['distance','uniform'], 'n_neighbors':[3,5,7,9,12,15,20,25]}
kn = GridSearchCV(knn, parameters,scoring= 'jaccard',
 cv=5)
kn.fit(X_train,y_train)
kn.best_params_
kn.best_score_
params = kn.best_params_
kn_grid = KNeighborsClassifier()
kn_grid.set_params(**params)
kn_grid.fit(X_train,y_train)

y_grid = kn_grid.predict(X_test)
generate_model_report(y_test,y_grid)


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={'weights':['distance','uniform'], 'n_neighbors':[3,5,7,9,12,15,20,25]}
kn = GridSearchCV(knn, parameters,scoring= 'accuracy',
 cv=5)
kn.fit(X_train,y_train)
kn.best_score_
params = kn.best_params_
kn_grid = KNeighborsClassifier()
kn_grid.set_params(**params)
kn_grid.fit(X_train,y_train)

y_grid = kn_grid.predict(X_test)
generate_model_report(y_test,y_grid)


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={'weights':['distance','uniform'], 'n_neighbors':[3,5,7,9,12,15,20,25]}
kn = GridSearchCV(knn, parameters,scoring= 'roc_auc',
 cv=5)
kn.fit(X_train,y_train)

params = kn.best_params_
kn_grid = KNeighborsClassifier()
kn_grid.set_params(**params)
kn_grid.fit(X_train,y_train)

y_grid = kn_grid.predict(X_test)
generate_model_report(y_test,y_grid)


# ROC Curve with the best parameters given by gridsearch especially for roc_auc scoring 

In [ ]:
generate_auc_roc_curve(kn_grid,X_test)


# SVM SUPPORT VECTOR MACHINE

In [ ]:
from sklearn import svm
svm = svm.SVC(class_weight = 'balanced')

In [ ]:
svm.fit(X_train,y_train)

In [ ]:
X_train.shape

In [ ]:
svm.get_params()

In [ ]:
y_svm = svm.predict(X_test)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(svm, X_test, y_test)  # doctest: +SKIP
plt.show()  # doctest: +SKIP

In [ ]:
from sklearn.metrics import classification_report
target=['class 0', 'class 1']
print(classification_report(y_test, y_svm, target_names=target))

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={"C":[1,10,100,500,1000],'kernel':["linear", "poly", "rbf", "sigmoid"],'gamma':['auto','scale']}
sv = GridSearchCV(svm, parameters,scoring= 'f1',
 cv=5)
sv.fit(X_train,y_train)
sv.best_params_
sv.best_score_

In [ ]:
generate_model_report(y_test,y_svm)

# SUPPORT VECTORS

In [ ]:
support_vector_indices = svm.support_
len(support_vector_indices)

In [ ]:
# Get number of support vectors per class
support_vectors_per_class = svm.n_support_
print(support_vectors_per_class)

In [ ]:
# Get support vectors themselves
support_vectors = svm.support_vectors_
support_vectors

In [ ]:
support_vectors.shape

> # DECISION TREES

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree1 = DecisionTreeClassifier()

In [ ]:
tree1.get_params()

In [ ]:
tree1.fit(X_train,y_train)

In [ ]:
y_tree = tree1.predict(X_test)

OVERFITTING

In [ ]:
tree1.score(X_train,y_train)

In [ ]:
tree1.score(X_test,y_test)

In [ ]:
generate_model_report(y_test,y_tree)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(tree1, X_test, y_test)  # doctest: +SKIP
plt.show()  # doctest: +SKIP

In [ ]:
generate_auc_roc_curve(tree1,X_test)

In [ ]:
disp = plot_precision_recall_curve(tree1, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: ')

># Evaluating after specifying the class weight as balanced.

In [ ]:
tree2 = DecisionTreeClassifier(class_weight = 'balanced')

In [ ]:
tree2.fit(X_train,y_train)
y_tree2 = tree2.predict(X_test)

In [ ]:
generate_model_report(y_test,y_tree2)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(tree2, X_test, y_test)  # doctest: +SKIP
plt.show()  # doctest: +SKIP

In [ ]:
disp = plot_precision_recall_curve(tree2, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: ')

In [ ]:
generate_auc_roc_curve(tree2,X_test)

> # FUNCTION FOR TREE REPORT

In [ ]:
def tree_report(clf):
    print("Depth is ", clf.get_depth())
    print("n_Leaves are:  ",clf.get_n_leaves())
#     print("The parameters are:" ,clf.get_params())
    print("Classes are:" , clf.classes_)
    print("The feature importances are: " , clf.feature_importances_)
    print("The inferred value of max_features:" , clf.max_features_)
    
    

In [ ]:
tree2.get_depth()

In [ ]:
tree2.get_n_leaves()

OVERFITTING!

In [ ]:
tree2.score(X_train, y_train)


In [ ]:
tree2.score(X_test,y_test)


In [ ]:
tree2.get_params()

In [ ]:
tree2.feature_importances_

> # GRID SEARCH CV WITH DIFFERENT SCORING METRICS FOR COMPARING TREE SCORE ON TEST DATA FOR LOW VARIANCE.

> # Starting with ccp_alpha = 0 (No cost complexity pruning)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={"max_depth":[15,20,30,35],'min_samples_split':[5,10,15,20]}
grid = GridSearchCV(tree1, parameters,scoring= 'accuracy',
 cv=10)
grid.fit(X_train,y_train)
params = grid.best_params_
tree_grid = DecisionTreeClassifier()
tree_grid.set_params(**params)
tree_grid.fit(X_train,y_train)
y_tree_grid = tree_grid.predict(X_test)
tree_grid.score(X_test,y_test)

In [ ]:
from sklearn import tree
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(tree_grid, feature_names=X_train, filled=True)

In [ ]:
tree_grid.score(X_train,y_train)

In [ ]:
grid.best_estimator_


In [ ]:
grid.best_estimator_.feature_importances_


In [ ]:
tree_grid.get_params()

> # #The alpha value controls overfitting as it is a pruning parameter.

# #ccp_alpha = 0.0001

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={"max_depth":[15,20,30,35],'min_samples_split':[5,10,15,20],'ccp_alpha':[0.0001]}
grid = GridSearchCV(tree1, parameters,scoring= 'recall',
 cv=10)
grid.fit(X_train,y_train)
params = grid.best_params_
tree_grid = DecisionTreeClassifier()
tree_grid.set_params(**params)
tree_grid.fit(X_train,y_train)
y_tree_grid = tree_grid.predict(X_test)
tree_grid.score(X_test,y_test)

In [ ]:
tree_grid.get_params()

In [ ]:
tree_report(tree_grid)

In [ ]:
tree_grid.score(X_train,y_train)

* >  The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the Gini importance.
> 
* >  Warning: impurity-based feature importances can be misleading for high cardinality features (many unique values). See sklearn.inspection.permutation_importance as an alternative.

In [ ]:
tree_grid.feature_importances_

# Permutation importance for feature evaluation

> The permutation importance of a feature is calculated as follows. 
* First, a baseline metric, defined by scoring, is evaluated on a (potentially different) dataset defined by the X.
* Next, a feature column from the validation set is permuted and the metric is evaluated again.
* The permutation importance is defined to be the difference between the baseline metric and metric from permutating the feature column.

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(tree_grid, X_train, y_train, n_repeats=10,
                                 random_state=0)
result.importances_mean


> # #ccp_alpha = 0.01

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={"max_depth":[15,20,30,35],'min_samples_split':[5,10,15,20],'ccp_alpha':[0.01]}
grid = GridSearchCV(tree1, parameters,scoring= 'balanced_accuracy',
 cv=10)
grid.fit(X_train,y_train)
params = grid.best_params_
tree_grid = DecisionTreeClassifier()
tree_grid.set_params(**params)
tree_grid.fit(X_train,y_train)
y_tree_grid = tree_grid.predict(X_test)
tree_grid.score(X_test,y_test)

In [ ]:
tree_grid.score(X_train,y_train)

> # #alpha = 0.01 and the depth is only 2! And the model is only taking one feature for training (obviously!), this shows the extent of pruning.

In [ ]:
tree_report(tree_grid)

# PLOTTING TREE

In [ ]:
from sklearn import tree
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(tree_grid, feature_names=X_train, filled=True)

In [ ]:
tree_grid.feature_importances_

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(tree_grid, X_train, y_train, n_repeats=10,
                                 random_state=0)
result.importances_mean

# # ccp_alpha = 0.1

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={"max_depth":[15,20,30,35],'min_samples_split':[5,10,15,20],'ccp_alpha':[0.1]}
grid = GridSearchCV(tree1, parameters,scoring= 'f1',
 cv=10)
grid.fit(X_train,y_train)
params = grid.best_params_
tree_grid = DecisionTreeClassifier()
tree_grid.set_params(**params)
tree_grid.fit(X_train,y_train)
y_tree_grid = tree_grid.predict(X_test)
tree_grid.score(X_test,y_test)

> * ACCURACY ON TRAINING DATA IS OBVIOUSLY THE LEAST THIS TIME BECAUSE IT DIDN'T EVEN FIT.
> * STILL THE TEST ACCURACY ID HIGH BUT IS NOT SIGNIFICANT AT ALL!!
> * LOOK OUT FOR THE TRAP, THE EVALUATION METRICS DO NOT GIVE YOU THE SIGNIFICANCE OF THEIR SCORES.
> * CHECK AND ANALYSE EVERY PARAMETER FOR FINAL EVALUATION.

In [ ]:
tree_grid.score(X_train,y_train)

In [ ]:
tree_report(tree_grid)

> * IT IS NOT TAKING ANY FEATURE INTO ACCOUNT, BECAUSE THE ENTIRE TREE GOT PRUNED!

In [ ]:
tree_grid.feature_importances_

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(tree_grid, X_train, y_train, n_repeats=10,
                                 random_state=0)
result.importances_mean

> * # alpha = 0.00001

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={"max_depth":[15,20,30,35],'min_samples_split':[5,10,15,20],'ccp_alpha':[0.00001]}
grid = GridSearchCV(tree1, parameters,scoring= 'recall',
 cv=10)
grid.fit(X_train,y_train)
params = grid.best_params_
tree_grid = DecisionTreeClassifier()
tree_grid.set_params(**params)
tree_grid.fit(X_train,y_train)
y_tree_grid = tree_grid.predict(X_test)
tree_grid.score(X_test,y_test)

In [ ]:
tree_report(tree_grid)

In [ ]:
tree_grid.score(X_train,y_train)

In [ ]:
tree_grid.feature_importances_

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(tree_grid, X_train, y_train, n_repeats=10,
                                 random_state=0)
result.importances_mean

 >#  choose alpha = 10^-3 or 10^-4

> # min_impurity_split float, default=0
* >  Threshold for early stopping in tree growth. A node will split if its impurity is above the threshold, otherwise it is a leaf.

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={"max_depth":[15,20,30,35],'min_samples_split':[5,10,15,20],'ccp_alpha':[0.00001]}
grid = GridSearchCV(tree1, parameters,scoring= 'f1',
 cv=10)
grid.fit(X_train,y_train)
params = grid.best_params_
tree_grid = DecisionTreeClassifier()
tree_grid.set_params(**params)
tree_grid.fit(X_train,y_train)
y_tree_grid = tree_grid.predict(X_test)
tree_grid.score(X_test,y_test)

In [ ]:
tree_grid.score(X_train,y_train)

In [ ]:
tree_report(tree_grid)

In [ ]:
from sklearn import tree
text_representation = tree.export_text(tree_grid)
print(text_representation)

> #  SET MAX_DEPTH: The maximum depth of the representation. 

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(tree_grid, feature_names=X_train, filled=True,max_depth = 4)

* > label: {‘all’, ‘root’, ‘none’}, optional (default=’all’)
* > Whether to show informative labels for impurity, etc. Options include ‘all’ to show at every node, ‘root’ to show only at the top root node, or ‘none’ to not show at any node

In [ ]:
fig = plt.figure(figsize=(25,20))
t = tree.plot_tree(tree_grid, feature_names=X_train, filled=True,max_depth = 10,label ='all',fontsize = 10 )

> # Permutation test score 
> * Calculates the p-value 
> * Calculates the classification score.

In [ ]:
from sklearn.model_selection import permutation_test_score
score, permutation_scores, pvalue = permutation_test_score(
    tree_grid, X_train, y_train, scoring="accuracy", cv=cv, n_permutations=100, n_jobs=1)
print("Classification score %s (pvalue : %s)" % (score, pvalue))

> #  LEARNING CURVE
> * With different scoring metrics.

In [ ]:
from yellowbrick.model_selection import LearningCurve
visualizer = LearningCurve(
    tree_grid, cv=10, scoring='f1',verbose = 0
)

visualizer.fit(X_train, y_train)        # Fit the data to the visualizer
visualizer.show()

In [ ]:
from yellowbrick.model_selection import LearningCurve
visualizer = LearningCurve(
    tree_grid, cv=10, scoring='precision',verbose = 0
)

visualizer.fit(X_train, y_train)        # Fit the data to the visualizer
visualizer.show()

In [ ]:
from yellowbrick.model_selection import LearningCurve
visualizer = LearningCurve(
    tree_grid, cv=10, scoring='recall',verbose = 0
)

visualizer.fit(X_train, y_train)        # Fit the data to the visualizer
visualizer.show()

In [ ]:
from yellowbrick.model_selection import LearningCurve
visualizer = LearningCurve(
    tree_grid, cv=10, scoring='accuracy',verbose = 0
)

visualizer.fit(X_train, y_train)        # Fit the data to the visualizer
visualizer.show()

> # PRECISION-RECALL CURVE

In [ ]:
disp = plot_precision_recall_curve(tree_grid, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: ')

> # ROC CURVE

In [ ]:
generate_auc_roc_curve(tree_grid,X_test)

> # CONFUSION MATRIX 

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(tree_grid, X_test, y_test)  # doctest: +SKIP
plt.show()  # doctest: +SKIP

> # CLASSIFICATION REPORT

In [ ]:
from sklearn.metrics import classification_report
target=['class 0', 'class 1']
print(classification_report(y_test, y_tree_grid, target_names=target))

> # EVALUATION METRICS
> * SCORES
> * LOSS

In [ ]:
generate_model_report(y_test,y_tree_grid)

> # BAGGING 

> # RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(oob_score = True, class_weight = "balanced")

In [ ]:
clf.fit(X_train,y_train)

>  PARAMETERS

In [ ]:
clf.get_params()

In [ ]:
y_clf = clf.predict(X_test)

In [ ]:
clf.score(X_test,y_test)

* >  OVERFITTING!!

In [ ]:
clf.score(X_train,y_train)

In [ ]:
generate_model_report(y_test,y_clf)

In [ ]:
def tree_report2(clf):
#     print("Depth is ", clf.get_depth())
#     print("n_Leaves are:  ",clf.get_n_leaves())
#     print("The parameters are:" ,clf.get_params())
    print("Classes are:" , clf.classes_)
    print("The feature importances are: " , clf.feature_importances_)
    print("oob score:",clf.oob_score_)
#     print("The inferred value of max_features:" , clf.max_features_)

In [ ]:
tree_report2(clf)

> # OOB_SCORE
* > Score of the training dataset obtained using an out-of-bag estimate.
* > This attribute exists only when oob_score is True.

In [ ]:
clf.oob_score_

* > Decision function computed with out-of-bag estimate on the training set. 
* > If n_estimators is small it might be possible that a data point was never left out during the bootstrap. 
* > In this case, oob_decision_function_ might contain NaN. 
* > This attribute exists only when oob_score is True.

In [ ]:
clf.oob_decision_function_

In [ ]:
clf.decision_path

> # clf1 
* > Changing ccp_aplha for pruning the tree and then we will evaluate the model. 
* > ccp_alpha = 0.0001

In [ ]:
clf1 = RandomForestClassifier(oob_score = True, class_weight = "balanced",ccp_alpha = 0.0001)
clf1.fit(X_train,y_train)
y_clf1 = clf1.predict(X_test)

In [ ]:
clf1.score(X_test,y_test)

In [ ]:
clf1.score(X_train,y_train)

In [ ]:
1-0.9982765612327656

> # The training accuracy reduced by 0.0017, test by  0.0005 BUT oob_score increased, this means we always need to check every parameter for the ultimate significance of any model.

In [ ]:
tree_report2(clf1)
#OOB_SCORE

In [ ]:
generate_model_report(y_test,y_clf1)

> # warm_start : bool, default=False
> # When set to True, reuse the solution of the previous call to fit and add more estimators to the ensemble,otherwise, just fit a whole new forest.
> # Let's evaluate after setting it to True.

In [ ]:
clf2 = RandomForestClassifier(oob_score = True, class_weight = "balanced",ccp_alpha = 0.0001, warm_start = True)
clf2.fit(X_train,y_train)
y_clf2 = clf2.predict(X_test)

In [ ]:
clf2.score(X_test,y_test)

In [ ]:
clf2.score(X_train,y_train)

In [ ]:
tree_report2(clf2)
#OOB_SCORE

In [ ]:
generate_model_report(y_test,y_clf2)

* > Using clf1 (warm_start = False) 

* > # Let's find the optimal max_depth, min_samples_split, min_samples_leaf, max_leaf_nodes.

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={"n_estimators":[50,100,150,200],"max_depth":[15,20,30,35],'min_samples_split':[5,10,15,20],'min_samples_leaf':[5,10]}
grid = GridSearchCV(clf1, parameters,scoring= 'balanced_accuracy', cv=10)
grid.fit(X_train,y_train)
params = grid.best_params_
gs = RandomForestClassifier(oob_score = True, class_weight = "balanced",ccp_alpha = 0.0001)
gs.set_params(**params)
gs.fit(X_train,y_train)
y_grid = gs.predict(X_test)
gs.get_params()

In [ ]:
gs.score(X_test,y_test)

In [ ]:
gs.score(X_train,y_train)

In [ ]:
gs.oob_score_

In [ ]:
generate_model_report(y_test,y_grid)

In [ ]:
tree_report2(gs)

In [ ]:
generate_auc_roc_curve(gs, X_test)

In [ ]:
from sklearn.metrics import classification_report
target=['class 0', 'class 1']
print(classification_report(y_test, y_grid, target_names=target))

> # Permutation Importance

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(gs, X_train, y_train, n_repeats=10,
                                 random_state=0)
result.importances_mean

> # PRECISION RECALL CURVE

In [ ]:
disp = plot_precision_recall_curve(gs, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: ')

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(gs, X_test, y_test)  # doctest: +SKIP
plt.show()  # doctest: +SKIP

In [ ]:
from sklearn.model_selection import permutation_test_score
score, permutation_scores, pvalue = permutation_test_score(
    gs, X_train, y_train, scoring="accuracy", cv=2, n_permutations=10, n_jobs=1)
print("Classification score %s (pvalue : %s)" % (score, pvalue))

In [ ]:
from yellowbrick.model_selection import LearningCurve
visualizer = LearningCurve( gs, cv=10, scoring='f1',verbose = 0 ) 
visualizer.fit(X_train, y_train)        # Fit the data to the visualizer 
visualizer.show()

In [ ]:
# fn=X_train
# cn=y_train
# fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)
# tree.plot_tree(gs.estimators_[0],
#                feature_names = fn, 
#                class_names=cn,
#                filled = True);

In [ ]:
gs.estimators_

> # Tree Visualization

In [ ]:
from sklearn import tree
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(gs.estimators_[0], feature_names=X_train, filled=True,max_depth = 2)

In [ ]:
from sklearn import tree
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(gs.estimators_[50], feature_names=X_train, filled=True,max_depth = 4)

> # BOOSTING

> # XGBOOST FOR CLASSIFICATION

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist'  )

In [ ]:
xgb.fit(X_train,y_train)

In [ ]:
xgb.get_params()

In [ ]:
y_xgb = xgb.predict(X_test)

In [ ]:
xgb.score(X_test,y_test)

> Overfitting

In [ ]:
xgb.score(X_train,y_train)

In [ ]:
generate_model_report(y_test,y_xgb)

In [ ]:
generate_auc_roc_curve(xgb,X_test)

In [ ]:
disp = plot_precision_recall_curve(xgb, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: ')

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(xgb, X_test, y_test)  # doctest: +SKIP
plt.show()  # doctest: +SKIP

In [ ]:
from yellowbrick.model_selection import LearningCurve
visualizer = LearningCurve(
    xgb, cv=10, scoring='f1',verbose = 0
)

visualizer.fit(X_train, y_train)        # Fit the data to the visualizer
visualizer.show()

> # To decide the value of gamma, we will take a look at the gain values (refer the formula)

In [ ]:
from xgboost import plot_tree
# rcParams['figure.figsize'] = 100,100
fig = plot_tree(xgb,num_trees=1)
plt.savefig("E:\test.png")
plt.close()
# gvz = xgb.to_graphviz(xgb_model, num_trees=xgb_model.best_iteration, rankdir=rankdir)


In [ ]:
parameters = {'learning_rate': [0.1,0.3, 0.5], #so called `eta` value
              'max_depth': [5,10,15,20],
              'min_child_weight': [0.25,0.5,1],              
              'subsample': [0.8,1],
              'n_estimators': [100,150,200,250,300],
               }


In [ ]:
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(xgb, parameters,scoring='precision', cv=5)
cv.fit(X_train,y_train)
params = cv.best_params_
xgb1 = XGBClassifier(tree_method = "hist")
xgb1.set_params(**params)

xgb1.fit(X_train,y_train)
y_grid = xgb1.predict(X_test)


In [ ]:
xgb1.get_params()

In [ ]:
xgb1.score(X_test,y_test)

In [ ]:
xgb1.score(X_train,y_train)

In [ ]:
generate_model_report(y_test,y_grid)

In [ ]:
from yellowbrick.model_selection import LearningCurve
visualizer = LearningCurve(
    xgb1, cv=10, scoring='f1',verbose = 0
)

visualizer.fit(X_train, y_train)        # Fit the data to the visualizer
visualizer.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(xgb, parameters,scoring='balanced_accuracy', cv=10)
cv.fit(X_train,y_train)
params = cv.best_params_
xgb2 = XGBClassifier(tree_method = "hist")
xgb2.set_params(**params)

xgb2.fit(X_train,y_train)
y_grid2 = xgb2.predict(X_test)